In [ ]:
import torch.nn
import os
import torch
import numpy as np
import pandas as pd
import csv
import glob
import sys
import math
import torch  
import torch.nn as nn
import time
class UNet_down_block(torch.nn.Module):
    def __init__(self, input_channel, output_channel, down_size):
        super(UNet_down_block, self).__init__()
        self.conv1 = torch.nn.Conv2d(input_channel, output_channel, 3, padding=1)
        self.bn1 = torch.nn.BatchNorm2d(output_channel)
        self.conv2 = torch.nn.Conv2d(output_channel, output_channel, 3, padding=1)
        self.bn2 = torch.nn.BatchNorm2d(output_channel)
        self.conv3 = torch.nn.Conv2d(output_channel, output_channel, 3, padding=1)
        self.bn3 = torch.nn.BatchNorm2d(output_channel)
        self.max_pool = torch.nn.MaxPool2d(2, 2)
        self.relu = torch.nn.ReLU()
        self.down_size = down_size
        self.elu = torch.nn.ELU()


    def forward(self, x):
        if self.down_size:
            x = self.max_pool(x)
        x = self.elu(self.bn1(self.conv1(x)))
        x = self.elu(self.bn2(self.conv2(x)))
        #x = self.relu(self.bn3(self.conv3(x)))
        return x

class UNet_up_block(torch.nn.Module):
    def __init__(self, prev_channel, input_channel, output_channel):
        super(UNet_up_block, self).__init__()
        #self.up_sampling = torch.nn.Upsample(scale_factor=2, mode='bilinear',align_corners=False)
        self.up_sampling = torch.nn.ConvTranspose2d(input_channel,input_channel, 2,stride = 2)
        self.conv1 = torch.nn.Conv2d(prev_channel + input_channel, output_channel, 3, padding=1)
        self.bn1 = torch.nn.BatchNorm2d(output_channel)
        self.conv2 = torch.nn.Conv2d(output_channel, output_channel, 3, padding=1)
        self.bn2 = torch.nn.BatchNorm2d(output_channel)
        self.conv3 = torch.nn.Conv2d(output_channel, output_channel, 3, padding=1)
        self.bn3 = torch.nn.BatchNorm2d(output_channel)
        self.relu = torch.nn.ReLU()
        self.elu = torch.nn.ELU()

    def forward(self, prev_feature_map, x):
        x = self.up_sampling(x)
        x = torch.cat((x, prev_feature_map), dim=1)
        x = self.elu(self.bn1(self.conv1(x)))
        x = self.elu(self.bn2(self.conv2(x)))
        #x = self.relu(self.bn3(self.conv3(x)))
        return x


class UNet(torch.nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        self.down_block1 = UNet_down_block(1, 8, False)
        self.down_block2 = UNet_down_block(8, 16, True)
        self.down_block3 = UNet_down_block(16, 32, True)
        self.down_block4 = UNet_down_block(32, 64, True)
        self.down_block5 = UNet_down_block(64, 128, True)

        self.mid_conv1 = torch.nn.Conv2d(128, 128, 3, padding=1)
        self.bn1 = torch.nn.BatchNorm2d(128)
        self.mid_conv2 = torch.nn.Conv2d(128, 128, 3, padding=1)
        self.bn2 = torch.nn.BatchNorm2d(128)
        self.mid_conv3 = torch.nn.Conv2d(128, 128, 3, padding=1)
        self.bn3 = torch.nn.BatchNorm2d(128)

        self.up_block1 = UNet_up_block(64, 128, 64)
        self.up_block2 = UNet_up_block(32, 64, 32)
        self.up_block3 = UNet_up_block(16, 32, 16)
        self.up_block4 = UNet_up_block(8, 16, 8)


        self.last_conv1 = torch.nn.Conv2d(8, 8, 3, padding=1)
        self.last_bn = torch.nn.BatchNorm2d(8)
        self.last_conv2 = torch.nn.Conv2d(8, 4, 1, padding=0)
        self.relu = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
        self.elu = torch.nn.ELU()

    def forward(self, x):
        self.x1 = self.down_block1(x)
        self.x2 = self.down_block2(self.x1)
        self.x3 = self.down_block3(self.x2)
        self.x4 = self.down_block4(self.x3)
        self.x5 = self.down_block5(self.x4)
        self.x5 = self.elu(self.bn1(self.mid_conv1(self.x5)))
        self.x5 = self.elu(self.bn2(self.mid_conv2(self.x5)))
        #self.x5 = self.relu(self.bn3(self.mid_conv3(self.x5)))
        x = self.up_block1(self.x4, self.x5)
        x = self.up_block2(self.x3, x)
        x = self.up_block3(self.x2, x)
        x = self.up_block4(self.x1, x)
        x = self.elu(self.last_bn(self.last_conv1(x)))
        x = self.last_conv2(x)
        return x

In [ ]:
class SteelDataset(torch.utils.data.Dataset):

	def __init__(self,folder, pathlist, labeled , Y_dict = None ,transform = None):

		self.labeled = labeled
		self.X_path = pathlist
		self.folder = folder
		if self.labeled:
			self.Y_dict = Y_dict
		#self.X_test_path = get_data_path(test_folder)

	def __len__(self):
		return len(self.X_path)

	def __getitem__(self, idx):
		path = self.X_path[idx]
		try:
			X = get_img(self.folder + '/' + path)
			img = torch.FloatTensor(X).view(1,256,1600)
		except:
			print(self.X_path)
			print(idx)
			print(self.X_path[idx])
			print(path)


		if self.labeled:
			labels = (np.zeros([4,256,1600]))
			if path in self.Y_dict:
				for classID in range(4):
					labels[classID,:,:] = rle2mask((self.Y_dict[path])[classID])
			label = torch.FloatTensor(labels).view(4,256,1600)
			return tuple([img , label])
		else:
			return tuple([img , path])


In [ ]:
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
 
def rle2mask(mask_rle, shape=(1600,256)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [ ]:
sigmoid = nn.Sigmoid()
def add_weight_decay(net, l2_value, skip_list=()):
	decay, no_decay = [], []
	for name, param in net.named_parameters():
		if not param.requires_grad: continue # frozen weights		            
		if len(param.shape) == 1 or name.endswith(".bias") or name in skip_list: no_decay.append(param)
		else: decay.append(param)
	return [{'params': no_decay, 'weight_decay': 0.}, {'params': decay, 'weight_decay': l2_value}]

def dice_channel_torch(probability, truth, threshold):
	batch_size = truth.shape[0]
	channel_num = truth.shape[1]
	mean_dice_channel = 0.
	with torch.no_grad():
		for i in range(batch_size):
			for j in range(channel_num):
				channel_dice = dice_single_channel(probability[i, j,:,:], truth[i, j, :, :], threshold[j])
				mean_dice_channel += channel_dice/(batch_size * channel_num)
	return mean_dice_channel


def dice_single_channel(probability, truth, threshold, eps = 1E-9):
	p = (probability.view(-1) > threshold).float()
	t = (truth.view(-1) > 0.5).float()
	dice = (2.0 * (p * t).sum() + eps)/ (p.sum() + t.sum() + eps)
	return dice

def run_length_decode(rle, height=256, width=1600, fill_value=1):
	mask = np.zeros((height,width), np.float32)
	if rle != '':
		mask=mask.reshape(-1)
		r = [int(r) for r in rle.split(' ')]
		r = np.array(r).reshape(-1, 2)
		for start,length in r:
			start = start-1  #???? 0 or 1 index ???
			mask[start:(start + length)] = fill_value
		mask=mask.reshape(width, height).T
	return mask

def run_length_encode(mask):
#possible bug for here
	m = mask.T.flatten()
	if m.sum()==0:
		rle=''
	else:
		m   = np.concatenate([[0], m, [0]])
		run = np.where(m[1:] != m[:-1])[0] + 1
		run[1::2] -= run[::2]
		rle = ' '.join(str(r) for r in run)
	return rle


def dice_loss( y_pred,y_true):
	weight = [1,1,1,1]
	smooth = 1e-9
	y_pred = sigmoid(y_pred)
	y_true_f = y_true.view(-1,4,256,1600)
	y_pred_f = y_pred.view(-1,4,256,1600)
	score = 0
	batch_size = y_true_f.shape[0]
	channel_num = y_true_f.shape[1]
	for i in range(batch_size):
		for j in range(channel_num):
			intersection = y_true_f[i,j,:,:] * y_pred_f[i,j,:,:] 
			score += weight[j] * ( (2. * intersection.sum() + smooth) / (y_true_f[i,j,:,:].sum() + y_pred_f[i,j,:,:].sum() + smooth) ) 
	score /= (batch_size )
	return - score

def bce_dice_loss( y_pred,y_true):
	return weighted_bceloss(y_pred,y_true) + dice_loss(y_pred,y_true)

def weighted_bceloss(y_pred,y_true):
	weight = [1,1,1,1]
	loss = 0
    
	for c in range(4):
		loss += bceloss(y_pred[:,c,:,:] ,y_true[:,c,:,:]) * weight[c] 
	return loss 

In [ ]:
#import pyvips
from PIL import Image
#from utils import *
#from Unet.py import *
def get_data_path(folder):
	X_path = []
	#print(os.listdir(folder))
	for filename in (os.listdir(folder)):
		#print(filename)
		X_path.append(filename)

	X_path = np.array(X_path)
	return X_path

def get_label(path):
	df = pd.read_csv(path)
	Ydict = {}
	for image_id in df.ImageId.unique():
		rle = [
			df.loc[ (df['ImageId']==image_id) & (df['ClassId'] == 1) , 'EncodedPixels'].values,
			df.loc[ (df['ImageId']==image_id) & (df['ClassId'] == 2) , 'EncodedPixels'].values,
			df.loc[ (df['ImageId']==image_id) & (df['ClassId'] == 3) , 'EncodedPixels'].values,
			df.loc[ (df['ImageId']==image_id) & (df['ClassId'] == 4) , 'EncodedPixels'].values
		]
		for classID in range(4):
			if len(rle[classID]) == 0:
				rle[classID] = ''
			else:
				rle[classID] = rle[classID][0]

		#print(y[0])
		#print(y[1])
		#print(y[2])
		Ydict[image_id ] =  rle
	del df
	return Ydict 


def get_img(path):
	with Image.open(path) as f:
		img = np.array(list(f.getdata()))
	f.close()
	#print('img shape',len(img),img[:,1])
	return img[:,1].reshape(256,1600)


def valid_test(epoch):
	model.eval()
	batch_size = 10
	Totaldice = 0
	Totalloss = 0
	for i , (batch_x,batch_y) in enumerate(validloader):
		batch_x = batch_x.cuda() 
		batch_y = batch_y.cuda()
		pV =  model(batch_x)
		loss = weighted_bceloss(pV,batch_y).item()
		#loss = bce_dice_loss(pV,batch_y).item() 
		dice = dice_channel_torch( sigmoid(pV) , batch_y , [0.5,0.5,0.5,0.5] )
		Totalloss += loss * len(batch_x)
		Totaldice += dice * len(batch_x)
	meandice = Totaldice / V
	meanloss = Totalloss / V
	#print(pV)
	print('epoch:{}   Valid: dice {:.3f} loss: {:.3f}'.format(epoch, meandice,meanloss )) #time.time()-start_time  
	model.train()
	return meanloss

In [ ]:
bceloss = nn.BCEWithLogitsLoss()
sigmoid = nn.Sigmoid()

train_folder = "../input/severstal-steel-defect-detection/train_images"
label_path = "../input/severstal-steel-defect-detection/train.csv"

X_train_path = get_data_path(train_folder)
Y_train_dict = get_label(label_path)

validDataset = SteelDataset(train_folder,X_train_path,labeled=1,Y_dict = Y_train_dict)

V = len(X_train_path)
validloader = torch.utils.data.DataLoader(validDataset , batch_size = 4, shuffle=False,num_workers=4)
model = torch.load("../input/model/model2.3.pth")
valid_test(0)

def Train(model):
	epochs = 3
	best = 100
	decay = 1e-4
	LR = 0.001
	#optim = torch.optim.Adam( add_weight_decay(model,decay) , lr= LR)
	optim = torch.optim.Adam( model.parameters(), lr= LR)
	print('train-------------lr:',LR ,'decay:',decay )
	model.cuda()
	for epoch in range(int(epochs)):
		model.train()
		for i , (batch_x , batch_y )in enumerate(trainloader):

			batch_x = batch_x.cuda()
			batch_y = batch_y.cuda()
			
			optim.zero_grad()
			y_hat = model(batch_x)
			loss = weighted_bceloss(y_hat,batch_y)
            #loss = bce_dice_loss(y_hat,batch_y)
			loss.backward()
			optim.step()

			if i % 100 == 99:
				loss = valid_test(epoch)
				if loss < best:
					best = loss
					print("saving model with loss: {:.3f}".format(loss))
					torch.save(model, 'model2.pth')

if __name__ == '__main__':
	#load = 0
	isGPU = torch.cuda.is_available()
	print ('PyTorch GPU device is available: {}'.format(isGPU))
	train_folder = "../input/severstal-steel-defect-detection/train_images"
	label_path = "../input/severstal-steel-defect-detection/train.csv"
	
	#train_folder , test_folder, label_path = sys.argv[1:]
	
	X_train_path = get_data_path(train_folder)
	Y_train_dict = get_label(label_path)
	
        
	#print(X_train_path)
	#print(Y_train_dict)
    
	V = 100
	X_valid_path = X_train_path[:V]
	X_train_path = X_train_path[V:]

	trainDataset = SteelDataset(train_folder,X_train_path,labeled=1,Y_dict = Y_train_dict)
	validDataset = SteelDataset(train_folder,X_valid_path,labeled=1,Y_dict = Y_train_dict)

	trainloader = torch.utils.data.DataLoader(trainDataset , batch_size = 4, shuffle=True,num_workers=4)
	validloader = torch.utils.data.DataLoader(validDataset , batch_size = 4, shuffle=False,num_workers=4)

	#model = UNet()
	model = torch.load("../input/model/model2.3.pth")
	Train(model)
	#Test(model)


def Train(model):
	epochs = 3
	best = 100
	decay = 1e-4
	LR = 0.001
	#optim = torch.optim.Adam( add_weight_decay(model,decay) , lr= LR)
	optim = torch.optim.Adam( model.parameters(), lr= LR)
	print('train-------------lr:',LR ,'decay:',decay )
	model.cuda()
	for epoch in range(int(epochs)):
		model.train()
		for i , (batch_x , batch_y )in enumerate(trainloader):

			batch_x = batch_x.cuda()
			batch_y = batch_y.cuda()
			
			optim.zero_grad()
			y_hat = model(batch_x)
			loss = weighted_bceloss(y_hat,batch_y)
            #loss = bce_dice_loss(y_hat,batch_y)
			loss.backward()
			optim.step()

			if i % 100 == 99:
				loss = valid_test(epoch)
				if loss < best:
					best = loss
					print("saving model with loss: {:.3f}".format(loss))
					torch.save(model, 'model2.pth')

if __name__ == '__main__':
	#load = 0
	isGPU = torch.cuda.is_available()
	print ('PyTorch GPU device is available: {}'.format(isGPU))
	train_folder = "../input/severstal-steel-defect-detection/train_images"
	label_path = "../input/severstal-steel-defect-detection/train.csv"
	
	#train_folder , test_folder, label_path = sys.argv[1:]
	
	X_train_path = get_data_path(train_folder)
	Y_train_dict = get_label(label_path)
	
        
	#print(X_train_path)
	#print(Y_train_dict)
    
	V = 100
	X_valid_path = X_train_path[:V]
	X_train_path = X_train_path[V:]

	trainDataset = SteelDataset(train_folder,X_train_path,labeled=1,Y_dict = Y_train_dict)
	validDataset = SteelDataset(train_folder,X_valid_path,labeled=1,Y_dict = Y_train_dict)

	trainloader = torch.utils.data.DataLoader(trainDataset , batch_size = 4, shuffle=True,num_workers=4)
	validloader = torch.utils.data.DataLoader(validDataset , batch_size = 4, shuffle=False,num_workers=4)

	#model = UNet()
	model = torch.load("../input/model/model2.3.pth")
	Train(model)
	#Test(model)


def Test(model):
	model.eval()
	#test_folder = "../input/severstal-steel-defect-detection/test_images"
	#X_test_path = get_data_path(test_folder)
	#X_test_path.sort()
	print("X_test size:{}".format(len(X_test_path)))
	print(X_test_path)
	testDataset = SteelDataset(test_folder,X_test_path,labeled=0)
	testloader = torch.utils.data.DataLoader(testDataset , batch_size = 4, shuffle=False,num_workers=4)
	outpath = 'submission.csv'
	f=open(outpath,"w")
	f.write("ImageId_ClassId,EncodedPixels\n")
	threshhold = [0.5,0.5,0.5,0.5]
	for i , (img,path) in enumerate(testloader):
		predlist = sigmoid(model(img.cuda()))
		#print(len(predlist))
		#print(predlist)
		for n in range(len(predlist)):
			#print(predlist.shape)
			pred = predlist[n]
			for ClassId in range(0,4):
				p = (pred[ClassId].flatten(1) > threshhold[ClassId]).float()
				encoded_value = mask2rle(p.cpu())
				#if len(encoded_value) > 0:
				f.write("{}_{},{}\n".format(path[n],ClassId + 1,encoded_value ))
	f.close()
	model.train()

if __name__ == '__main__':
    test_folder = "../input/severstal-steel-defect-detection/test_images"
    X_test_path = get_data_path(test_folder)
    testDataset = SteelDataset(test_folder,X_test_path,labeled=0)
    testloader = torch.utils.data.DataLoader(testDataset , batch_size = 4, shuffle=False,num_workers=4)

    model = torch.load("../input/model/model2.3.pth")
    Test(model)

import matplotlib.pyplot as plt
#model = torch.load("../input/model/model2.3.pth")
fig, axs = plt.subplots(5, figsize=(12, 12))

sample_img = get_img('../input/severstal-steel-defect-detection/train_images/0025bde0c.jpg')
#predlist = sigmoid(model(torch.from_numpy(sample_img.astype('float32')).view(-1,1,256,1600).cuda()))

label_path = "../input/severstal-steel-defect-detection/train.csv"
path = "0025bde0c.jpg"
Y_dict = get_label(label_path)

print(Y_dict[path])
labels = (np.zeros([4,256,1600]))
for classID in range(4):
	rle = (Y_dict[path])[classID]
	print(rle)
	labels[classID,:,:] = rle2mask( (Y_dict[path])[classID] )
    
predlist = labels

#print(predlist)
for n in range(len(predlist)):
	print(predlist.shape)
	pred = predlist[n]
	for ClassId in range(0,4):
		print(labels[ClassId].sum())
		#p = (pred[ClassId].flatten(1) > 0.5).float()
		#img = np.array(p.cpu()).reshape(256,1600)
		#encoded_value = mask2rle(p.cpu())
				#if len(encoded_value) > 0:
		#img = rle2mask(encoded_value)
        #print(img)
		axs[ClassId+1].imshow(pred)
		axs[ClassId+1].axis('off')
axs[0].imshow(sample_img)
axs[0].axis('off')